In [150]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder

In [151]:
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful AI Chatbot and you talk with user in {language} langugae. User's name is {name}"),
    HumanMessagePromptTemplate.from_template("User will talk friendly to Chatbot"),
    MessagesPlaceholder(variable_name="messages")
])

In [152]:
load_dotenv()

True

In [153]:
llm = ChatGroq(temperature=0, model='llama3-8b-8192')

In [154]:
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages, Sequence

In [155]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str
    name: str

In [156]:
def model(state: State):
    msg = prompt_template.invoke({"messages": state['messages'], "language": state['language'], "name": state["name"]})
    response = llm.invoke(msg)
    return {"messages": [response]}

In [157]:
from langgraph.graph import StateGraph
from langgraph.graph import START, END
from langgraph.checkpoint.memory import MemorySaver

In [158]:
graph = StateGraph(state_schema=State)

graph.add_edge(START, "model")
graph.add_node("model", model)

memory = MemorySaver()

app = graph.compile(checkpointer=memory)

In [159]:
config = {"configurable": {"thread_id": "user123"}}
query = "My name is Prabal"
language  = "hindi"
name = "Pro"
input_msg = [HumanMessage(query)]

In [160]:
app.invoke({"messages": input_msg, "language": language, "name": name}, config)

{'messages': [HumanMessage(content='My name is Prabal', additional_kwargs={}, response_metadata={}, id='ad0eb722-db38-4955-a39c-9c720e687447'),
  AIMessage(content='Prabal ji! मैं आपका स्वागत करता हूँ! (I welcome you!) क्या आप मुझसे कुछ बातचीत करना चाहते हैं? (What do you want to talk about?)', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 60, 'total_tokens': 113, 'completion_time': 0.054902021, 'prompt_time': 0.007400627, 'queue_time': 0.272584883, 'total_time': 0.062302648}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_8b7c3a83f7', 'finish_reason': 'stop', 'logprobs': None}, id='run--57947061-0774-4f47-b547-cdd63e3a8454-0', usage_metadata={'input_tokens': 60, 'output_tokens': 53, 'total_tokens': 113})],
 'language': 'hindi',
 'name': 'Pro'}